## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hamminga (`np.Hmming`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:
        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("literki.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/literki.bmp --no-check-certificate
if not os.path.exists("wzorA.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/wzorA.bmp --no-check-certificate



In [ ]:
size = 21
hamming_window = np.hamming(size)
hamming_2d = np.outer(hamming_window, hamming_window)

FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(size))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, size]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(size))
FSpaceColsM = np.outer(np.ones([1, size]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR <= 0.1
# FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

filter_result = FilterFI * hamming_2d


figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, filter_result, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
figFilter.show()

In [ ]:
img = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
f_img = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
f_img_shift = np.fft.fftshift(f_img, [0,1])


right = img.shape[1] - size
down = img.shape[0] - size
filter_result_padd = cv2.copyMakeBorder(filter_result, down//2, down - down//2, right//2, right - right//2, cv2.BORDER_CONSTANT, value=0)

f_filter = cv2.dft(np.float32(filter_result_padd), flags=cv2.DFT_COMPLEX_OUTPUT)
f_filter_shift = np.fft.fftshift(f_filter, [0,1])

# mag = cv2.magnitude(f_filter_shift[:, :, 0], f_filter_shift[:, :, 1])

filt = np.abs(f_filter_shift)

mag = cv2.magnitude(filt[:, :, 0], filt[:, :, 1])

FilterF3 = np.repeat(mag[:, :, np.newaxis], 2, axis=2)

img = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
f_img = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
f_img_shift = np.fft.fftshift(f_img, [0,1])

In [ ]:

mult_result = f_img_shift * FilterF3
img_shift = np.fft.ifftshift(mult_result, [0, 1])
idft_img = cv2.idft(img_shift, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
idft_img = np.absolute(idft_img)
ifft_img = cv2.magnitude(idft_img[:, :, 0], idft_img[:, :, 1])
res = np.round(ifft_img).astype('uint8')

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(20, 15)


axs[0].imshow(img, 'gray', vmin=0, vmax=256)
axs[0].axis('off')
axs[0].set_title("Obraz")
axs[1].imshow(mag, 'gray')
axs[1].axis('off')
axs[1].set_title("Filtr")
axs[2].imshow(res, 'gray')
axs[2].axis('off')
axs[2].set_title("Rezultat")
